# Test - Compile mc_reach.pyx and build shared object file in Google Colab environment  


In [ ]:
import os
import sys
import subprocess
import glob
import pathlib

# logic to accomodate Google Colab
try:
    import google.colab

    ENV_IS_CL = True

    root = pathlib.Path("/content/t-route").resolve()

    subprocess.run(
        [
            "git",
            "clone",
            "https://github.com/NOAA-OWP/t-route.git",
        ]
    )

except:
    ENV_IS_CL = False
    print("This example notebook is only intended to work in the Google Colab environment.")

# define directory variables and append path
fortran_source_dir = os.path.join(root, "src", "fortran_routing", "mc_pylink_v00", "MC_singleSeg_singleTS")
sys.path.append(fortran_source_dir)

routing_v02_dir = os.path.join(root, "src", "python_routing_v02", "fast_reach")
sys.path.append(routing_v02_dir)

reservoir_source_dir = os.path.join(root, "src", "fortran_routing", "mc_pylink_v00", "Reservoir_singleTS")
sys.path.append(reservoir_source_dir)

COMPILE_cython = True
if COMPILE_cython:

    # compile vaPrecision.f90
    subprocess.run(
        [
            "gfortran", 
            "varPrecision.f90", 
            "-c", 
            "-O0", 
            "-fPIC"],
        cwd=fortran_source_dir,
    )

    # compile MCsingleSegStime_f2py_NOLOOP.f90
    subprocess.run(
        [
            "gfortran",
            "-c",
            "-O0",
            "-fPIC",
            "-o",
            "mc_single_seg.o",
            "MCsingleSegStime_f2py_NOLOOP.f90"
        ],
        cwd=fortran_source_dir,
    )

    # compile pyMCsingleSegStime_NoLoop.f90
    subprocess.run(
        [
            "gfortran",
            "pyMCsingleSegStime_NoLoop.f90",
            "-c",
            "-o",
            "pymc_single_seg.o",
            "-O3",
            "-fPIC"
        ],
        cwd=fortran_source_dir,
    )

    # copy *.o files to python_routing_v02/fast_reach 
    for obj_file in glob.glob(os.path.join(fortran_source_dir, "*.o")):
        print(obj_file)
        subprocess.run(["cp", obj_file, routing_v02_dir])    

    # compile vaPrecision.f90
    subprocess.run(
        [
            "gfortran", 
            "varPrecision.f90", 
            "-c", 
            "-O0", 
            "-fPIC"],
        cwd=reservoir_source_dir,
    )
    
    # compile module_levelpool.f90
    subprocess.run(
        [
            "gfortran",
            "-c",
            "-O2",
            "-fPIC",
            "-o",
            "module_levelpool.o",
            "module_levelpool.f90"
        ],
        cwd=reservoir_source_dir,
    )
    
    # compile pymodule_levelpool.f90
    subprocess.run(
        [
            "gfortran",
            "-c",
            "-O2",
            "-fPIC",
            "-o",
            "pymodule_levelpool.o",
            "pymodule_levelpool.f90"
        ],
        cwd=reservoir_source_dir,
    ) 

    # copy *.o files to python_routing_v02/fast_reach 
    for obj_file in glob.glob(os.path.join(reservoir_source_dir, "*.o")):
        print(obj_file)
        subprocess.run(["cp", obj_file, routing_v02_dir])

    numpy_I = "/usr/local/lib/python3.6/dist-packages/numpy/core/include"
    py_I = "/usr/include/python3.6"
    py_lib = "/usr/include/python3.6" 

    # for unknown reasons this line does not work with subprocess.run
    %cd "/content/t-route/src/python_routing_v02/fast_reach"
    ! cython -3 -v -p --gdb --line-directives -Wextra --cleanup 3 fortran_wrappers.pxd *.pyx

    file = ["fortran_wrappers","mc_reach","reservoir","reach","utils"]
    for f in file:
        subprocess.run(
          [
              "gcc",
              "-pthread",
              "-Wno-unused-result",
              "-Wsign-compare",
              "-DNDEBUG",
              "-fwrapv",
              "-O2",
              "-Wall",
              "-Wstrict-prototypes",
              "-fPIC",
              "-fno-strict-aliasing",
              "-I",
              numpy_I,
              "-I",
              py_I,
              "-mtune=generic",
              "-c",
              "-o",
              f + ".o",
              f + ".c"
          ],
          cwd=routing_v02_dir,
        )

    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-shared", 
            "-L",
            py_lib, 
            "-lgfortran",
            "-o",
            "reach.cpython-36m-x86_64-linux-gnu.so",
            "mc_single_seg.o",
            "pymc_single_seg.o",
            "fortran_wrappers.o",
            "reach.o"
        ],
        cwd=routing_v02_dir,
    )

    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-shared", 
            "-L",
            py_lib, 
            "-lgfortran",
            "-o",
            "reservoir.cpython-36m-x86_64-linux-gnu.so",
            "module_levelpool.o",
            "pymodule_levelpool.o",
            "fortran_wrappers.o",
            "reservoir.o"
        ],
        cwd=routing_v02_dir,
    )

    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-shared", 
            "-L",
            py_lib, 
            "-o",
            "mc_reach.cpython-36m-x86_64-linux-gnu.so",
            "mc_reach.o"
        ],
        cwd=routing_v02_dir,
    )

    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-shared", 
            "-L",
            py_lib, 
            "-o",
            "utils.cpython-36m-x86_64-linux-gnu.so",
            "utils.o"
        ],
        cwd=routing_v02_dir,
    )    

import mc_reach